In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import os
from copy import deepcopy
import random 
import cv2

In [2]:
dt = {}
for i in range(6):
    for j in range(6):
        dt[str(i)+str(j)] = str

#load dataframe
df1 = pd.read_csv(r"C:\Users\ABHINAV\Downloads\mlware23\train_faces.csv", dtype = dt)
df2 = pd.read_csv(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\train\train_landmarks.csv", dtype= dt)

df3 = pd.read_csv(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\sample_prediction.csv", dtype= dt)

df1.head()

,image,00,01,02,03,04,05,10,11,12,...,42,43,44,45,50,51,52,53,54,55
0,hwJUKjziH0.jpg,22,20,14,4,43,40,35,53,25,...,31,41,13,32,33,10,2,30,50,44
1,smzyIVQ4Ve.jpg,3,50,43,14,5,13,33,52,1,...,41,35,45,30,44,22,51,0,32,15
2,IT3H2QbaJY.jpg,55,25,43,10,33,20,22,13,54,...,0,42,11,41,12,23,30,51,35,32
3,Yceag6Ca0B.jpg,12,32,52,5,15,42,55,4,10,...,34,20,13,21,22,45,33,41,23,35
4,nDBcp5ZyPj.jpg,33,44,12,45,50,54,20,24,0,...,55,34,4,25,52,15,11,21,2,32


In [3]:
columns = df1.columns.tolist()
for i in columns[1:]:
    df1[i] = df1[i].replace('0', '00')
    df1[i] = df1[i].replace('1', '01')
    df1[i] = df1[i].replace('2', '02')
    df1[i] = df1[i].replace('3', '03')
    df1[i] = df1[i].replace('4', '04')
    df1[i] = df1[i].replace('5', '05')

df1.head()

,image,00,01,02,03,04,05,10,11,12,...,42,43,44,45,50,51,52,53,54,55
0,hwJUKjziH0.jpg,22,20,14,04,43,40,35,53,25,...,31,41,13,32,33,10,02,30,50,44
1,smzyIVQ4Ve.jpg,03,50,43,14,05,13,33,52,01,...,41,35,45,30,44,22,51,00,32,15
2,IT3H2QbaJY.jpg,55,25,43,10,33,20,22,13,54,...,00,42,11,41,12,23,30,51,35,32
3,Yceag6Ca0B.jpg,12,32,52,05,15,42,55,04,10,...,34,20,13,21,22,45,33,41,23,35
4,nDBcp5ZyPj.jpg,33,44,12,45,50,54,20,24,00,...,55,34,04,25,52,15,11,21,02,32


val, train split on the dataframes

In [4]:
n1 = len(df1)
n2 = len(df2)
val_split = 0.2
val_df1 = df1[:int(n1*val_split)]
train_df1 = df1[int(n1*val_split):]

val_df2 = df2[:int(n2*0.2)]
train_df2 = df2[int(n2*0.2):]

In [5]:
print(train_df1.shape)

(2456, 37)


partitions image to 36*36 tiles

In [6]:
def load_images1(df):
    ret = []
    for idx in tqdm(df.index):
        pieces = []
        img_name = df.loc[idx, "image"]
        img = Image.open(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\train\faces" + chr(92)+img_name)
        img = np.array(img)
        for i in range(6):
            for j in range(6):
                pieces.append(img[i*50:(i+1)*50, j*50:(j+1)*50])
        ret.append(pieces)
    return np.array(ret)

def load_images2(df):
    ret = []
    for idx in tqdm(df.index):
        pieces = []
        img_name = df.loc[idx, "image"]
        img = Image.open(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\train\landmarks" + chr(92)+img_name)
        img = np.array(img)
        for i in range(6):
            for j in range(6):
                pieces.append(img[i*50:(i+1)*50, j*50:(j+1)*50])
        ret.append(pieces)
    return np.array(ret)
# newly_added
def load_images(new_images):
    ret = []
    for i in range(len(new_images)):
        pieces = []
        img = new_images[i]
        for i in range(6):
            for j in range(6):
                pieces.append(img[i*50:(i+1)*50, j*50:(j+1)*50])
        ret.append(pieces)
    return np.array(ret)

In [7]:
train_img1 = load_images1(train_df1)
val_img1 = load_images1(val_df1)

train_img2 = load_images2(train_df2)
val_img2 = load_images2(val_df2)


100%|██████████| 600/600 [00:01<00:00, 551.51it/s]


In [9]:
"""train_img1 = train_img1/255
train_img2 = train_img2/255
val_img1 = val_img1/255
val_img2 = val_img2/255"""

'train_img1 = train_img1/255\ntrain_img2 = train_img2/255\nval_img1 = val_img1/255\nval_img2 = val_img2/255'

Generates labels to each tile of image 

In [8]:
def load_labels(df):
    ret = []
    for idx in tqdm(df.index):
        label = np.zeros(shape=(36,36))
        for i in range(6):
            for j in range(6):
                place = i*6 + j
                val = int(df.loc[idx, str(i)+str(j)][0])*6 + int(df.loc[idx, str(i)+str(j)][1])
                label[place][val] = 1
        ret.append(label)
    return np.array(ret)


train_labels1 = load_labels(train_df1)
val_labels1 = load_labels(val_df1)

train_labels2 = load_labels(train_df2)
val_labels2 = load_labels(val_df2)



100%|██████████| 600/600 [00:00<00:00, 1288.14it/s]


In [9]:
print(train_img1.shape)

(2456, 36, 50, 50, 3)


In [10]:
print(train_labels1[0].shape)

(36, 36)


model description

In [13]:

inp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once

#here we only define the layers and feed the input later
conv1 = tf.keras.layers.Conv2D(filters = 10,kernel_size= (5,5), strides=5, padding='same')
#max_pooling1=tf.keras.layers.MaxPool2D(pool_size=5,strides=5)
conv2 = tf.keras.layers.Conv2D(10, (5,5), strides=5, padding='same')
#max_pooling2=tf.keras.layers.MaxPool2D(pool_size=5,strides=5)
conv3 = tf.keras.layers.Conv2D(10, (5,5), strides=5, padding='same')
flat = tf.keras.layers.Flatten()

drop = tf.keras.layers.Dropout(.5)

#convLSTM = tf.keras.layers.ConvLSTM2D(10, (5,5), padding='same',  activation='tanh', recurrent_activation='hard_sigmoid', input_shape=(50,50,3))
dense1 = tf.keras.layers.Dense(1296, kernel_regularizer=tf.keras.regularizers.l2(0.001))

dense2 = tf.keras.layers.Dense(1296, kernel_regularizer=tf.keras.regularizers.l2(0.001))

#now we feed each puzzle piece one by one to the layers and store the encodings
mid = []
for i in range(36):
    x = conv1(inp[:,i]) #first dimension is batch_size
    #x = max_pooling1(x)
    x = conv2(x)
    x = conv3(x)
    
    #x = max_pooling2(x)
    x = flat(x)
    
    mid.append(x)
print(x.shape)
x = tf.keras.layers.concatenate(mid, axis=-1)
print(x.shape)
x = dense1(x)
drop
x = dense2(x)
print(x.shape)
x = tf.keras.layers.Reshape((36,36))(x) #the output is reshaped into a 36x36 output
x = tf.keras.layers.Softmax(axis=-1)(x) #softmax to calculate probability

"\ninp = tf.keras.layers.Input(shape=(36,50,50,3)) #input all puzzle pieces at once\n\n#here we only define the layers and feed the input later\nconv1 = tf.keras.layers.Conv2D(filters = 10,kernel_size= (5,5), strides=5, padding='same')\n#max_pooling1=tf.keras.layers.MaxPool2D(pool_size=5,strides=5)\nconv2 = tf.keras.layers.Conv2D(10, (5,5), strides=5, padding='same')\n#max_pooling2=tf.keras.layers.MaxPool2D(pool_size=5,strides=5)\nconv3 = tf.keras.layers.Conv2D(10, (5,5), strides=5, padding='same')\nflat = tf.keras.layers.Flatten()\n\ndrop = tf.keras.layers.Dropout(.5)\n\n#convLSTM = tf.keras.layers.ConvLSTM2D(10, (5,5), padding='same',  activation='tanh', recurrent_activation='hard_sigmoid', input_shape=(50,50,3))\ndense1 = tf.keras.layers.Dense(1296, kernel_regularizer=tf.keras.regularizers.l2(0.001))\n\ndense2 = tf.keras.layers.Dense(1296, kernel_regularizer=tf.keras.regularizers.l2(0.001))\n\n#now we feed each puzzle piece one by one to the layers and store the encodings\nmid = []\

In [14]:
import numpy as np
import tensorflow as tf
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.applications import EfficientNetB0
from vit_keras import vit


base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(None, 300, 300, 3))


model = vit.vit_b16(
    image_size=300,
    activation='softmax',
    pretrained=True,
    include_top=False,
    pretrained_top=False,
    classes=36*36,
    transformer_layers=12,
    transformer_dropout=0.1,
    transformer_attention_dropout=0.1,
    transformer_hidden_size=768,
    transformer_num_heads=12,
    name='vit'
)

for layer in base_model.layers:
    layer.trainable = False

concat = tf.keras.layers.Concatenate()([base_model.output, model.output])

output = Flatten()(concat)
output = Dense(36*36, activation='softmax')(output)

final_model = tf.keras.models.Model(inputs=base_model.input, outputs=output)

final_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

x_train = train_img1
y_train = train_labels1
x_val = val_img1
y_val = val_labels1



final_model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

(None, 36)
(None, 1296)


In [15]:
model1 = tf.keras.Model(inputs=inp, outputs=x)
model1.compile(optimizer="RMSprop", loss="categorical_crossentropy", metrics=["accuracy"])

model2 = tf.keras.Model(inputs=inp, outputs=x)
model2.compile(optimizer="RMSprop", loss="categorical_crossentropy")


In [16]:
print((mid))

[<KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTensor: shape=(None, 36) dtype=float32 (created by layer 'dense_2')>, <KerasTenso

In [17]:
from keras.callbacks import ModelCheckpoint
model_cp = ModelCheckpoint(filepath="best.h5", monitor="val_accuracy", verbose=1)

model trained over faces

In [ ]:
model1.fit(x=train_img1, 
        y=train_labels1, 
        batch_size=16, 
        validation_data=[val_img1, val_labels1], 
        shuffle=True,
        epochs = 5)

model trained over landmarks

In [49]:
model2.fit(x=train_img2, 
        y=train_labels2, 
        batch_size=16, 
        validation_data=[val_img2, val_labels2], 
        shuffle=True,
        epochs=5)

Epoch 1/5
150/150 [==============================] - 17s 69ms/step - loss: 3.6501 - val_loss: 3.5013
Epoch 2/5
150/150 [==============================] - 7s 48ms/step - loss: 3.4823 - val_loss: 3.4974
Epoch 3/5
150/150 [==============================] - 7s 47ms/step - loss: 3.4526 - val_loss: 3.4357
Epoch 4/5
150/150 [==============================] - 7s 49ms/step - loss: 3.4315 - val_loss: 3.4518
Epoch 5/5
150/150 [==============================] - 7s 49ms/step - loss: 3.4210 - val_loss: 3.4300


In [39]:
faces_lis = os.listdir(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\test\faces")
landmarks_lis = os.listdir(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\test\landmarks")

In [40]:
df4 = pd.DataFrame()
df4.insert(0, "image", faces_lis)

df5 = pd.DataFrame()
df5.insert(0, "image", landmarks_lis)

In [41]:
def load_images_facest(df):
    ret = []
    for idx in tqdm(df.index):
        pieces = []
        img_name = df.loc[idx, "image"]
        img = Image.open(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\test\faces" + chr(92)+img_name)
        img = np.array(img)
        for i in range(6):
            for j in range(6):
                pieces.append(img[i*50:(i+1)*50, j*50:(j+1)*50])
        ret.append(pieces)
    return np.array(ret)

def load_images_landmarkst(df):
    ret = []
    for idx in tqdm(df.index):
        pieces = []
        img_name = df.loc[idx, "image"]
        img = Image.open(r"C:\Users\ABHINAV\Downloads\mlware23\dataset\test\landmarks" + chr(92)+img_name)
        img = np.array(img)
        for i in range(6):
            for j in range(6):
                pieces.append(img[i*50:(i+1)*50, j*50:(j+1)*50])
        ret.append(pieces)
    return np.array(ret)

In [42]:
img_facest = load_images_facest(df4)
img_landmarkst = load_images_landmarkst(df5)

100%|██████████| 1096/1096 [00:01<00:00, 870.16it/s]


skip

In [24]:
pred1 = model1.predict(val_img1)
pred1 = np.argmax(pred1, axis=-1)

pred2 = model2.predict(val_img2)
pred2 = np.argmax(pred2, axis=-1)

19/19 [==============================] - 2s 42ms/step


In [43]:
pred4 = model1.predict(img_facest)
pred4 = np.argmax(pred4, axis=-1)

pred5 = model2.predict(img_landmarkst)
pred5 = np.argmax(pred5, axis=-1)

35/35 [==============================] - 3s 42ms/step


skip

skip


In [49]:
pred_list1 = []
for i in range(pred1.shape[0]):
    t = []
    for j in range(pred1[i].shape[0]):
        t.append(str(pred1[i][j]//6) + str(pred1[i][j]%6))
    pred_list1.append(t)

image_df1 = pd.DataFrame(val_df1["image"])
label_df1 = pd.DataFrame(pred_list1)
out1 = pd.concat([image_df1, label_df1], axis=1)
out1.columns = df1.columns

skip

In [50]:
pred_list2 = []
for i in range(pred2.shape[0]):
    t = []
    for j in range(pred2[i].shape[0]):
        t.append(str(pred2[i][j]//6) + str(pred2[i][j]%6))
    pred_list2.append(t)

image_df2 = pd.DataFrame(val_df2["image"])
label_df2 = pd.DataFrame(pred_list2)
out2 = pd.concat([image_df2, label_df2], axis=1)
out2.columns = df2.columns

In [44]:
pred_list3 = []
for i in range(pred4.shape[0]):
    t = []
    for j in range(pred4[i].shape[0]):
        t.append(str(pred4[i][j]//6) + str(pred4[i][j]%6))
    pred_list3.append(t)

image_df3 = pd.DataFrame(df4["image"])
label_df3 = pd.DataFrame(pred_list3)
out3 = pd.concat([image_df3, label_df3], axis=1)
out3.columns = df1.columns

pred_list4 = []
for i in range(pred5.shape[0]):
    t = []
    for j in range(pred5[i].shape[0]):
        t.append(str(pred5[i][j]//6) + str(pred5[i][j]%6))
    pred_list4.append(t)

image_df4 = pd.DataFrame(df5["image"])
label_df4 = pd.DataFrame(pred_list4)
out4 = pd.concat([image_df4, label_df4], axis=1)
out4.columns = df2.columns

In [45]:
out3.head()

,image,00,01,02,03,04,05,10,11,12,...,42,43,44,45,50,51,52,53,54,55
0,00iOHCZTws.jpg,05,22,53,05,55,32,42,53,53,...,43,00,21,00,50,01,32,00,05,13
1,03fxX7jYxI.jpg,24,55,52,50,05,10,00,50,55,...,55,00,40,05,00,53,55,55,05,54
2,05evYM76RD.jpg,34,00,52,34,13,15,21,04,30,...,55,42,00,05,31,00,04,21,35,30
3,08UaxmNJtO.jpg,52,55,21,55,55,55,50,43,40,...,22,50,55,52,55,50,52,30,55,40
4,09gb5hcslV.jpg,55,55,30,55,43,22,42,53,42,...,32,43,55,34,55,55,55,22,42,50


In [46]:
def prediction_to_submission(df):
    df = df.copy()
    for s in ['r','c']:
        for i in range(6):
            for j in range(6):
                df[str(s)+str(i)+str(j)] = None

    for idx in df.index:
        for i in range(6):
            for j in range(6):
                df.loc[idx, 'r'+str(i)+str(j)] = int(df.loc[idx, str(i)+str(j)][0])
                df.loc[idx, 'c'+str(i)+str(j)] = int(df.loc[idx, str(i)+str(j)][1])

    columns_to_remove = []
    for i in range(6):
        for j in range(6):
            columns_to_remove.append(str(i)+str(j))

    return df.drop(columns=columns_to_remove)
submission1 = prediction_to_submission(out3)
submission2 = prediction_to_submission(out4)

submission = pd.concat([submission1, submission2])
submission.head()

,image,r00,r01,r02,r03,r04,r05,r10,r11,r12,...,c42,c43,c44,c45,c50,c51,c52,c53,c54,c55
0,00iOHCZTws.jpg,0,2,5,0,5,3,4,5,5,...,3,0,1,0,0,1,2,0,5,3
1,03fxX7jYxI.jpg,2,5,5,5,0,1,0,5,5,...,5,0,0,5,0,3,5,5,5,4
2,05evYM76RD.jpg,3,0,5,3,1,1,2,0,3,...,5,2,0,5,1,0,4,1,5,0
3,08UaxmNJtO.jpg,5,5,2,5,5,5,5,4,4,...,2,0,5,2,5,0,2,0,5,0
4,09gb5hcslV.jpg,5,5,3,5,4,2,4,5,4,...,2,3,5,4,5,5,5,2,2,0


In [47]:
print(submission.shape)

(1996, 73)


In [48]:
submission.to_csv("opt.csv", index = False)